In [1]:
import numpy as np 
import scipy.stats as st 
from queue import *
import math
import pandas as pd

In [2]:
from distances import *


In [3]:

class Event: 
    def __init__( self, time ):
        self.time = time   
    def __lt__( self, other ):
        return self.time < other.time  
    def handle( self ):
        """updates the global simulation clock time to the class's current time attribute
        preconditions: none"""
        global simulationclock
        simulationclock = self.time
    
class NewRequest(Event, object): #An arrival occurs 
    def __init__( self, time):
        """creates a new object of class NewRequest and sets the class time attribute to the time argument
        Preconditions: time is the desired simulation clock time of the NewRequest"""
        super(NewRequest, self).__init__( time )
    def __lt__( self, other ):
        return self.time < other.time   
    def handle( self ):
        """Updates the global simulation clock time to self.time, creates a request object with a 
        random buiness center and the current time, and schedules the next arrival
        If there are no mechanics at dispatch, it puts the request in a queue
        If there are mechanics at dispatch, it decreases the number of mechanics by 1 and schedules a mechanic
        arrival event"""
        super(NewRequest, self).handle() # inherit operations in the function in the base class, simulation clock updated to the time when the arrival event occurs
        center = getBC()
        request = Request(center, simulationclock, None )
        eventlist.put( NewRequest( simulationclock + arrivaltime() ) )
        if DispatchCenter.num_mechs > 0: 
            DispatchCenter.num_mechs -= 1 
            dist = distances.at['Dis', request.center.name]
            travel_time = dist/60
            eventlist.put(mech_Arrives(simulationclock+travel_time, request.center, request)) 
        else:  
            requestQueue.put(request)

class mech_Arrives(Event, object): 
    def __init__(self, time, business_center, request): 
        super(mech_Arrives, self).__init__(time)
        self.business_center = business_center
        self.request = request
    def __lt__( self, other ):
        return self.time < other.time  
    def handle(self):
        """update the global simulation time to the event time, schedule the end of the diagnosis for 
        diagnosis_time later, and increase the number of busy mechanics at the business center by 1"""
        super(mech_Arrives, self).handle()
        self.request.time_responded = (simulationclock - self.request.timeEntered)*60
        time_to_respond_avg.append(self.request.time_responded) if simulationclock>t else None
        diag_time = diagnosis_Time(self.business_center)
        self.business_center.number_of_mechanics += 1 
        eventlist.put(diag_ends(simulationclock+ diag_time, self.business_center))
        
        

class diag_ends(Event, object):
    def __init__(self, time, business_center):
        super(diag_ends, self).__init__(time)
        self.business_center = business_center
    def __lt__( self, other ):
        return self.time < other.time  
    def handle(self):
        super(diag_ends, self).handle()
        BC_prob = np.random.uniform(0,1)
        options = ['repair', 'replace']
        outcome_probabilities = np.array([0.8195, 0.1805])
        outcome = np.random.choice(options, 1, p = outcome_probabilities)[0]   
        if outcome == 'repair':
            eventlist.put(repair_start(simulationclock, self.business_center))
        else:
            eventlist.put(call_van(simulationclock, self.business_center))

class repair_start(Event, object): 
    def __init__(self, time, business_center): 
        super(repair_start, self).__init__(time)
        self.business_center = business_center
    def __lt__( self, other ):
        return self.time < other.time  
    def handle(self): 
        super().handle()
        repair_time = st.beta.rvs(2.5, 7.5, -0.0077049330279426054, 1.557915557823419, size=1)[0]

#         repair_time = st.beta.rvs(2.6158007964976218, 7.460626334186162, -0.0077049330279426054, 1.557915557823419, size=1)[0]
        eventlist.put(repair_end(simulationclock+ repair_time, self.business_center))


class repair_end(Event, object):
    def __init__(self, time, business_center): 
        super(repair_end, self).__init__(time)
        self.business_center = business_center
    def __lt__( self, other ):
        return self.time < other.time  
    def handle(self): 
        super(repair_end, self).handle()
        eventlist.put(mechanic_leaves_business_center(simulationclock, self.business_center))           

            
class mech_goes_to_dispatch(Event,object): 
    def __init__(self, time):
        super(mech_goes_to_dispatch,self).__init__(time)
    def __lt__( self, other ):
        return self.time < other.time  
    def handle(self):
        super(mech_goes_to_dispatch,self).handle()
        DispatchCenter.num_mechs += 1
        if not requestQueue.empty():
            req = requestQueue.get()
            dist = distances.at['Dis', req.center.name]
            travel_time = dist/60
            eventlist.put(mech_Arrives(simulationclock+travel_time, req.center, req))
            DispatchCenter.num_mechs -= 1
            

        


####### REPLAACEMENT STARTS HERE
class call_van(Event,object):
    def __init__(self, time, business_center):
        super().__init__( time )
        self.business_center = business_center
    def handle(self):
        super(call_van,self).handle()
        copier_request =  Request(self.business_center, simulationclock, None)
        if DispatchCenter.available_vans > 0: 
            DispatchCenter.available_vans -= 1
            
            dist = distances.at['Dis',self.business_center.name]
            travel_time = dist/60
            eventlist.put(van_arrives_at_bc(simulationclock+travel_time, copier_request.center, copier_request))           
        else:
            copierQueue.put(copier_request)
        eventlist.put(mechanic_leaves_business_center(simulationclock, self.business_center))           


class mechanic_leaves_business_center(Event, object):
    def __init__(self, time, business_center):
        super().__init__( time )
        self.business_center = business_center
    def handle(self):
        if requestQueue.empty():
                dist= distances.at['Dis', self.business_center.name]
                travel_time = dist/60
                eventlist.put(mech_goes_to_dispatch(simulationclock+travel_time))
                self.business_center.number_of_mechanics -= 1 
        else: 
            req = requestQueue.get()
            dist = distances.at[req.center.name, self.business_center.name]
            travel_time = dist/60
            eventlist.put(mech_Arrives(simulationclock+travel_time, req.center, req))
            self.business_center.number_of_mechanics -= 1 

            
          
class van_arrives_at_bc(Event, object):
    def __init__(self, time, business_center, copier_request ):
        super().__init__( time )
        self.business_center = business_center
        self.copier_request = copier_request
    def handle(self):
        super(van_arrives_at_bc, self).handle()
        eventlist.put(van_returns_to_dispatch(simulationclock+swaptime_customer(), self.business_center, self.copier_request))
        


class van_returns_to_dispatch(Event, object):
    def __init__(self, time, business_center, copier_request):
        super().__init__(time)
        self.business_center = business_center
        self.copier_request = copier_request
    def handle(self):
        super(van_returns_to_dispatch, self).handle()
        self.copier_request.time_responded = (simulationclock - self.copier_request.timeEntered)*60
        copier_req_times.append(self.copier_request.time_responded) if simulationclock>t else None
        dist = distances.at['Dis', self.business_center.name]
        travel_time = dist/60 
        eventlist.put(van_swaps_copiers(simulationclock+travel_time))  

                      
class van_swaps_copiers(Event, object):
    def __init__(self, time):
        super().__init__(time)
    def handle(self):
        super(van_swaps_copiers, self).handle()
        eventlist.put(vans_finishes_swap(simulationclock + swaptime_dispatch()))
            
class vans_finishes_swap(Event, object):
    def __init__(self, time):
        super().__init__(time)
    def handle(self):
        super(vans_finishes_swap , self).handle()
        if copierQueue.empty():
            DispatchCenter.available_vans += 1
        else:
            cop_req = copierQueue.get()
            dist = distances.at['Dis', cop_req.center.name]
            travel_time = dist/60
            eventlist.put(van_arrives_at_bc(simulationclock+travel_time, cop_req.center, cop_req))
            
    

class Request(object): 
    def __init__( self, center, timeEntered, time_responded=None ):
        self.center = center
        # self.id = id      
        self.timeEntered = timeEntered
        self.time_responded = time_responded

def arrivaltime(): #Comes from input analysis question 6  
    global arrivalrate
    time_of_day = simulationclock%24
    if time_of_day < 22 and time_of_day >3: 
        arrivalrate = (-0.067*(time_of_day**2)) + 1.581*(time_of_day) - 1.289 
    else: 
        arrivalrate = 1.137 
        
    return np.random.exponential( 1 / arrivalrate )

def swaptime_dispatch(): #Service time to swap at the dispatch center, keeping everything in hours 
    return np.random.triangular(left=10/60, mode=15/60, right=25/60)

def swaptime_customer(): #Service time to swap at customer loctation 
    return np.random.triangular(left=20/60, mode=30/60, right = 1)
        
def diagnosis_Time(bc):
    if bc.name in ['BC_2', 'BC_3', 'BC_9']:
        return 1/60*np.random.normal(16.13, 2.89)
    else: 
        return 1/60*np.random.normal(22.15, 5.34)


  



    

In [4]:
class business_center(object):
    def __init__( self, name,number_of_mechanics=0, number_of_vans=0):
        self.name=name
        self.number_of_mechanics = number_of_mechanics
        self.number_of_vans = number_of_vans

In [5]:
class dispatch(object):
    def __init__(self, available_vans, num_mechs):
        self.available_vans = available_vans
        self.num_mechs = num_mechs


In [6]:
bc_names = ['BC_1', 'BC_2', 'BC_3', 'BC_4', 'BC_5', 'BC_6', 'BC_7', 'BC_8', 'BC_9', 'BC_10']

def getBC():
    BC_prob = np.random.uniform(0,1)
    probabilities = np.array([0.039509, 0.082089, 0.107916, 0.135418, 0.118386, 
                              0.055005, 0.123830, 0.057937, 0.137512, 0.142398])
  
    return np.random.choice(bc_list, 1, p = probabilities)[0]

mec_avgs=[]
van_avgs=[]
t=100
T=200
B=100
for i in range(B):
    simulationclock = 0
    requestQueue = Queue()
    copierQueue = Queue()
    avail_vans = 3
    num_mechs = 11
    DispatchCenter = dispatch( avail_vans, num_mechs)
    eventlist = PriorityQueue()
    a = NewRequest( arrivaltime() )
    eventlist.put( a )
    time_to_respond_avg = []
    copier_req_times = []
    bc_list = []
    for i in range(len(bc_names)):
        bc_list.append(business_center(bc_names[i], 0, 0))
    while simulationclock < T:
        e = eventlist.get()
        e.handle()

    mech_avg = sum(time_to_respond_avg)/len(time_to_respond_avg) 
    copier_req_times = sum(copier_req_times)/len(copier_req_times)

    
    mec_avgs.append(np.mean(np.array(time_to_respond_avg)))
    van_avgs.append(np.mean(np.array(copier_req_times)))
    



In [7]:
np.mean(np.array(mec_avgs))

53.68338651780343

In [8]:
np.mean(np.array(van_avgs))

140.30569156697405